In [ ]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from datetime import timezone
import numpy as np
import plotly.express as px

DSI Lab: Task 1

This excercise combines multiple phases of the CRISP-DM from Data Collection to Modelling and Model Evaluation. You should apply your knowledge on all the steps in the
process and also iterate through multiple steps.
The dataset is a collection of environmental sensor data from the H-Building of FH-Aachen.

## 1. Task a & b

Read '07092021bimsim_roomparquet'

In [ ]:
path = './datasets/07092021bimsim_room.parquet'
df = pd.read_parquet(path)
print(df.head(10))
print(df.info())


Convert date columns ('id'=unix-ts, 'datetime'=string)

In [ ]:
df['datetime_converted'] = pd.to_datetime(df['datetime'])
df['id_converted'] = pd.to_datetime(df['id'], unit='s')
print(df.info())
print(df.describe())

## 2. Task c

Compare the timestamps in the two respective columns. What is their difference?
Could this be relevant? (Hint: Consider the capturing process)


In [ ]:
dfd = df[['datetime','id_converted','datetime_converted']]
print(dfd.head(10))
print(df['datetime_converted'].tail(5))
print(df['id_converted'].tail(5))

It seems like 'datetime' is equal to 'id' with an offset of +1 hour. 'id' is coded as unix-timestamp and has more time information then datetime, in case one of the columns should be dropped, its better to kick out the one with less information. But there are timestamps, where the difference is bigger then one hour. So 'id' does not 100% relate to 'datetime'.

## 3. Task d

Are there columns that can be safely dropped for the following steps?


In [ ]:
# Either 'sensor' or 'source' might be dropped, aslong as there values stay equal
print('Drop source: ' + str(df['sensor'].equals(df['source'])))

def sensorFitsRoom():
    df_sensor_room = df[['sensor','room']]
    unique_sensors = df['sensor'].unique()
    # print(unique_sensors)
    for sensor in unique_sensors:
        result =  df_sensor_room.loc[df['sensor'] == sensor]
        if result['room'].unique().size > 1:
            return False
    return True

# 'room' might be dropped, if there is only a one-to-one connection between sensor and room
print('Drop room: ' + str(sensorFitsRoom()))

def datetimeEqualsId():
    id_converted = df['id_converted'].dt.floor(freq='s')
    pd.to_datetime(df['id'], unit='s')
    datetime_shifted = df['datetime_converted'] - pd.Timedelta(hours=1)
    return id_converted.equals(datetime_shifted)

# 'datetime' might be dropped, if it equals 'id'
print('Drop datetime: ' + str(datetimeEqualsId()))

## 4. Task e

Are there outliers in the data?


In [ ]:
values = df[['celsius','pressure','humidity']]
print(values.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))) # https://stackoverflow.com/a/47207283

fig = px.box(df,x='celsius')
fig.update_layout(height=300, width=600)
fig.show()

fig = px.box(df,x='pressure')
fig.update_layout(height=300, width=600)
fig.show()

fig = px.box(df,x='humidity')
fig.update_layout(height=300, width=600)
fig.show()

The minimum 'celsius' is '-149.528' whereas the mean is '21.2777'. We know that the data is from sensors in rooms. So we could be sure, that negative temperature values are outliers. Also is the maximum 'humidity' at '97.0518' percent. This is also quite high for a room. Even when its closed.
The plot also show, that there are some outliers in 'celsius', 'pressure' and 'humidity'.

## 5. Task f,g

Group the data roomwise and print some statistics!
Create line plots for the rooms using plotly!

In [ ]:
df_rooms = df.groupby(by='room')
print(df_rooms.describe().apply(lambda s: s.apply(lambda x: format(x, 'g'))))

fig = px.line(df, x='datetime', y='celsius', color='room')
fig.show()

fig = px.line(df, x='datetime', y='pressure', color='room')
fig.show()

fig = px.line(df, x='datetime', y='humidity', color='room')
fig.show()

## 6. Task h

Create a 3D scatterplot for two rooms using plotly! Can you see differences in the data?

In [ ]:
df_rooms = df.loc[df['room'].isin(['H203', 'H106'])]
fig = px.scatter_3d(df_rooms, x='celsius', y='humidity', z='pressure', color='room')
fig.update_layout(width=1000, height=800)

fig.show()